# Sprint 5: Interactive Dashboard

## Colab Setup:


In [10]:
# Display + plotting
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.3f}'.format

# Set up dataset (get from kagglehub)
import kagglehub
path = kagglehub.dataset_download("jiashenliu/515k-hotel-reviews-data-in-europe")
print("Path to dataset files:", path)

import plotly as py
import plotly.express as px
import plotly.graph_objs as go

import math

Path to dataset files: /Users/simritlidhar/.cache/kagglehub/datasets/jiashenliu/515k-hotel-reviews-data-in-europe/versions/1


In [11]:
df = pd.read_csv(f"{path}/Hotel_Reviews.csv")
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.700,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.900,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.361,4.916
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.700,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.500,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.361,4.916
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.700,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.100,"[' Leisure trip ', ' Family with young childre...",3 days,52.361,4.916
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.700,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.800,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.361,4.916
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.700,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.700,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.361,4.916


## Part A: Data Card (updated)

**Source & Link:** 515K Hotel Reviews Data in Europe (Kaggle) - https://www.kaggle.com/datasets/jiashenliu/515k-hotel-reviews-data-in-europe/data

**Shape:** 515,738 rows x 18 columns

**Time Coverage:** August 4, 2015 - August 3, 2017



##Column Dictionary:
- **Hotel_Address:** address of the hotel
- **Additional_Number_of_Scoring:** number of scores given without an accompanying review
- **Review_Date:** date when the review was posted
- **Average_Score:** average score of the hotel
- **Hotel_Name:** name of the hotel
- **Reviewer_Nationality:** nationality of the reviewer
- **Negative_Review:** text content of negative review, if the reviewer did not give a negative review it says "No Negative"
- **Review_Total_Negative_Word_Counts:** number of words in negative review
- **Total_Number_of_Reviews:** the total number of reviews
- **Positive_Review:** text content of positve reviews, if the reviewer did not give a positive review it says "No Positive"
- **Review_Total_Positive_Word_Counts:** number of words in positive review
- **Total_Number_of_Reviews_Reviewer_Has_Given:** number of reviews given by the reviewer
- **Reviewer_Score:** score given by the reviewer
- **Tags:** tags given to hotel by the reviewer                         
- **days_since_review:** days between date of review and the date it was scraped        
- **lat:** latitude of hotel
- **lng:** longitude of hotel

New Columns:
- **Hotel_Country:** country of the reviewed hotel
- **Year:** year when review was posted
- **Review_Total_Word_Counts:** the total word-counts derived from adding positive review word-counts and negative review word-counts




**Key ID Columns:** 'Hotel_Name', 'Review_Date'


##Missingness Snapshot:
N/A


##Quirks:
- **Hotel_Country** only includes 6 countries: Netherlands, UK, France, Spain, Italy, Austria


In [12]:
#run before every plotly graph
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

## B. Transformation


In [13]:
from datetime import datetime
df['Review_Date'] = pd.to_datetime(df['Review_Date'], errors='coerce')

df['Year'] = df['Review_Date'].dt.year
df['Month'] = df['Review_Date'].dt.month
df['Month_Name'] = df['Review_Date'].dt.month_name()

def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

df = df.rename(columns={'Review_Total_Positive_Word_Counts': 'Pos_Word_Count',
                        'Review_Total_Negative_Word_Counts': 'Neg_Word_Count'})

df['Season'] = df['Month'].apply(get_season)
df['Season'] = pd.Categorical(df['Season'], categories=['Spring', 'Summer', 'Fall', 'Winter'], ordered=True)

df['Hotel_Country'] = df['Hotel_Address'].str.split().str[-1]
df['Hotel_Country'] = df['Hotel_Country'].replace('Kingdom', 'United Kingdom')

# Reviewer bias indicator
df['Reviewer_Difference'] = df['Reviewer_Score'] - df['Average_Score']

# Drop extreme or missing values for smooth visuals
df = df.dropna(subset=['Reviewer_Score', 'Average_Score', 'lat', 'lng'])
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Neg_Word_Count,Total_Number_of_Reviews,Positive_Review,...,Tags,days_since_review,lat,lng,Year,Month,Month_Name,Season,Hotel_Country,Reviewer_Difference
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.700,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.361,4.916,2017,8,August,Summer,Netherlands,-4.800
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.700,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.361,4.916,2017,8,August,Summer,Netherlands,-0.200
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.700,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,...,"[' Leisure trip ', ' Family with young childre...",3 days,52.361,4.916,2017,7,July,Summer,Netherlands,-0.600
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.700,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,...,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.361,4.916,2017,7,July,Summer,Netherlands,-3.900
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.700,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,...,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.361,4.916,2017,7,July,Summer,Netherlands,-1.000


In [14]:
import panel as pn
import panel.widgets as pnw
#!pip install jupyter_bokeh
#%pip install nbformat

# --- Widgets ---
country_select = pnw.MultiChoice(
    name="Select Country",
    options=sorted(df['Hotel_Country'].unique().tolist()),
    solid=True
)

year_slider = pnw.IntSlider(
    name="Select Year",
    start=int(df['Year'].min()),
    end=int(df['Year'].max()),
    step=1,
    value=int(df['Year'].min())
)

score_slider = pnw.RangeSlider(
    name="Reviewer Score Range",
    start=float(df['Reviewer_Score'].min()),
    end=float(df['Reviewer_Score'].max()),
    value=(5.0, 10.0)
)

season_select = pnw.Select(
    name="Select Season",
    options=['All'] + sorted(df['Season'].unique().tolist()),
    value='All'
)



In [15]:
def filtered_df(countryList, year, score_range, season):
    data = df.copy()

    # Apply filters
    if countryList:
        data = data[data['Hotel_Country'].isin(countryList)]
    if season != 'All':
        data = data[data['Season'] == season]

    data = data[
        (data['Year'] == year) &
        (data['Reviewer_Score'].between(score_range[0], score_range[1]))
    ]
    return data


In [16]:
df_filtered = pn.bind(filtered_df, country_select, year_slider, score_slider, season_select)

In [17]:
pn.extension("plotly", "tabulator")

def plot_overview(country_select, year_slider, score_slider, season_select):
    data = filtered_df(country_select, year_slider, score_slider, season_select)
    if data.empty:
        return pn.pane.Markdown("**No data available for selected filters.**")
    
    configure_plotly_browser_state()
    #get the average score of the hotels
    hotel_level = data.groupby(['Hotel_Name', 'Hotel_Country'], as_index=False).agg({
        'Average_Score': 'mean',
        'Total_Number_of_Reviews': 'first'
    })
    country_avg = hotel_level.groupby('Hotel_Country', as_index=False).agg({
        'Average_Score': 'mean',
        'Total_Number_of_Reviews': 'sum'
    })

    fig = px.bar(
        country_avg.sort_values('Average_Score', ascending = False),
        x = 'Hotel_Country',
        y = 'Average_Score',
        color = 'Average_Score',
        hover_data = {'Total_Number_of_Reviews':True, 'Average_Score':':.2f'},
        title="Average Hotel Review Score by Country",
        labels={
            'Hotel_Country': 'Country',
            'Average_Score': 'Average Review Score'
        },
        color_continuous_scale="Blues"
    )
    return fig

panel_plot_overview = pn.bind(plot_overview, country_select, year_slider, score_slider, season_select)

def score_over_time(country_select):
    configure_plotly_browser_state()
    if country_select:
        yearly_scores = df[df['Hotel_Country'].isin(country_select)].groupby(['Year', 'Hotel_Country'], as_index=False)['Reviewer_Score'].mean()
    else:
        yearly_scores = df.groupby(['Year', 'Hotel_Country'], as_index=False)['Reviewer_Score'].mean()

    fig = px.line(
        yearly_scores,
        x='Year',
        y='Reviewer_Score',
        color='Hotel_Country',
        title='Reviewer Scores Over Time by Country',
        labels={
            'Reviewer_Score':'Review Score'
        }
    )
    fig.update_xaxes(
        tickmode='linear',
        dtick=1,
        tick0=yearly_scores.Year.min()
    )
    return fig

panel_score_over_time = pn.bind(score_over_time, country_select)

def plot_word_counts(country_select, year_slider, score_slider, season_select):
    data = filtered_df(country_select, year_slider, score_slider, season_select)
    if data.empty:
        return pn.pane.Markdown("**No data available for selected filters.**")
    
    configure_plotly_browser_state()
    country_sentiment = data.groupby('Hotel_Country', as_index=False).agg({
        'Pos_Word_Count': 'mean',
        'Neg_Word_Count': 'mean'
    })

    trace1 = go.Bar(
                    x = country_sentiment.Hotel_Country,
                    y = country_sentiment.Pos_Word_Count.round(2),
                    name = "Positive",
                    text = country_sentiment.Pos_Word_Count.round(2))
    trace2 = go.Bar(
                    x = country_sentiment.Hotel_Country,
                    y = country_sentiment.Neg_Word_Count.round(2),
                    name = "Negative",
                    text = country_sentiment.Neg_Word_Count.round(2))
    fig = go.Figure()
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_layout(
        barmode='group',
        title='Average Positive vs. Negative Word Counts per Review by Country',
        xaxis_title='Country',
        yaxis_title='Average Word Count',
        legend_title='Sentiment',
        template='plotly_white'
    )
    return fig

panel_plot_word_counts = pn.bind(plot_word_counts, country_select, year_slider, score_slider, season_select)


def country_by_season(country_select, year_slider, score_slider, season_select):
    data = filtered_df(country_select, year_slider, score_slider, season_select)
    if data.empty:
        return pn.pane.Markdown("**No data available for selected filters.**")
    
    country_names = []
    country_data = []
    
    for country in data['Hotel_Country'].unique().tolist():
        country_names.append(country)
        country_data.append(data[data['Hotel_Country'] == country].groupby('Season', observed=True).Reviewer_Score.agg('mean'))
    country_by_season = pd.DataFrame(country_data, index=country_names)
    country_by_season_long = pd.melt(country_by_season.reset_index(), id_vars='index', var_name='Season', value_name='Rating')
    country_by_season_long
    
    fig = px.bar(country_by_season_long, x='Season', y='Rating', color='index', facet_row='index')
    fig.update_yaxes(range=[math.floor(country_by_season_long['Rating'].min() * 2) / 2, math.ceil(country_by_season_long['Rating'].max() * 2) / 2])
    fig.update_layout(
        height=60+90*len(country_names), 
        title_text=f"Average Review Score per Season in {year_slider}, by Hotel Country",
        xaxis_title='Season',
        yaxis_title='Rating',
        legend_title='Hotel Country')
    
    for annotation in fig.layout.annotations:
        annotation.text = ""
    
    return fig

panel_country_by_season = pn.bind(country_by_season, country_select, year_slider, score_slider, season_select)

def table_view(country_select, year_slider, score_slider, season_select):
    data = filtered_df(country_select, year_slider, score_slider, season_select)
    return pn.widgets.Tabulator(data[['Hotel_Name', 'Reviewer_Score', 'Average_Score', 'Season', 'Hotel_Country']],
                                pagination='remote', page_size=10, height=300)
panel_table_view = pn.bind(table_view, country_select, year_slider, score_slider, season_select)


In [19]:
filters = pn.Column(
    "### Filters",
    country_select,
    year_slider,
    score_slider,
    season_select,
    width=300
)

main = pn.Column(
    "## European Hotel Review Dashboard",
    pn.Row(panel_plot_overview),
    pn.Row(panel_score_over_time),
    pn.Row(panel_plot_word_counts),
    pn.Row(panel_country_by_season),
    pn.Row(panel_table_view),
)

#dashboard = pn.Row(filters, main)
#dashboard.servable()  # for `panel serve app.py`
# OR:
dashboard.save("hotel_dashboard.html", embed=True)
dashboard


  0%|                             | 0/15 [00:00<?, ?it/s]

  7%|█▍                   | 1/15 [00:03<00:45,  3.25s/it]

 13%|██▊                  | 2/15 [00:04<00:29,  2.25s/it]

 27%|█████▌               | 4/15 [00:07<00:16,  1.53s/it]

 33%|███████              | 5/15 [00:08<00:14,  1.50s/it]

...

 67%|█████████████▎      | 10/15 [00:17<00:07,  1.56s/it]

 73%|██████████████▋     | 11/15 [00:19<00:07,  1.82s/it]

 87%|█████████████████▎  | 13/15 [00:22<00:03,  1.53s/it]

 93%|██████████████████▋ | 14/15 [00:23<00:01,  1.46s/it]

Row
    [0] Column(width=300)
        [0] Markdown(str)
        [1] MultiChoice(name='Select Country', options=['Austria', 'France', ...])
        [2] IntSlider(end=2017, name='Select Year', start=2015, value=2015)
        [3] RangeSlider(end=10.0, name='Reviewer Score Range', start=2.5, value=(5.0, 10.0), value_end=10.0, value_start=5.0)
        [4] Select(options=['All', 'Fall', ...], value='All')
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
        [2] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
        [3] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
        [4] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
        [5] Row
            [0] ParamFunction(function, _pane=Tabulator, defer_load=False)